In [1]:
eval_path  ="."

In [2]:
import os
import pymolviz as pmv
import numpy as np
import pandas as pd
from linecache import getline

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np



def get_out_file_information(path, position_of_sulfur_in_xyz_file=15, max_opt_cycle=1000, save=''):

    out_file = '/TS_search_opt.out'
    coords_file = '/reactants.xyz'
    runtime_file = '/time_elapsed.txt'

    phrase = "GEOMETRY OPTIMIZATION CYCLE"
    ts_phase = "<= TS"
    ci_phase = "<= CI"
    frequ_match = "***imaginary mode***"
    reactants_phase = "Image     E(Eh)   dE(kcal/mol)  max(|Fp|)  RMS(Fp)"

    total_cycles = {}

    for conf in os.listdir(path):
        
 
        conf_details = []
        file = open(path+conf+out_file, 'r').readlines()


        # get sulfur atom coordinates
        sulfur_coord = (open(path+conf+coords_file, 'r').readlines())[position_of_sulfur_in_xyz_file-1].split()
        if sulfur_coord[0] == 'S':
            conf_details.append((float(sulfur_coord[1]), float(sulfur_coord[2]), float(sulfur_coord[3])))
        else:
            print(conf, sulfur_coord)

        # get number of opt cycles
        matches = [match for match in file if phrase in match]
        if matches == []:
            conf_details.append(0)
            print(conf, 'has no opt cycles -> value set to 0')
        else:
            if int(matches[-1].split()[4]) == max_opt_cycle:
                print(conf, f'has {max_opt_cycle} opt cycles')
            conf_details.append(int(matches[-1].split()[4]))

        # get energy delta of TS to reactants and system energy in Hartrees
        ts_match = [match for match in file if ts_phase in match]
        if ts_match != []:
            conf_details.append(float(ts_match[0].split()[2])) #kcal/mol
            conf_details.append(float(ts_match[0].split()[1])) #Hartrees
        else:
            conf_details.append(0)

        # get reactants energy system energy in Hartrees
        react_match = [ind for (ind, match) in  enumerate(file, 1) if reactants_phase in match]
        if react_match != []:
            conf_details.append(float(file[react_match[0]].split()[1])) #Hartrees
        else:
            conf_details.append(0)

        # get energy delta of CI to reactants
        ci_match = [match for match in file if ci_phase in match]
        if ci_match != []:
            conf_details.append(float(ci_match[0].split()[3]))
        else:
            conf_details.append(0)

        # get negative vibtrational frequencies -> Idee: Alles die mehr als 1 negativen Wert haben Auffälligkeiten/Merkwürdige Strukturen im Pathway 
        # -> lässt sich erstmal nicht erklären/bestätigen (s_spherical)
        freq_match = [float(match.split()[1]) for match in file if frequ_match in match]
        if freq_match != []:
            conf_details.append(freq_match)
        else:
            conf_details.append([0])

        # get run-time information
        exists = os.path.exists(path+conf+runtime_file)
        if exists:
            runtime = (open(path+conf+runtime_file, 'r').read())
            conf_details.append(int(runtime))
        else:
            conf_details.append(0)

        total_cycles[conf] = conf_details

    if save != '':
        with open(f'{eval_path}{save}_cycle_values', 'w') as f:
            f.write('conf;sulfur_atom_coordinates;opt_cycles;ts_dE_kcal_mol;ts_Hartees;reactants_Hartees;ci_dE_kcal_mol;vibrational_frequencies;runtime\n')
            for item in zip(total_cycles.keys(), total_cycles.values()):
                f.write(str(item[0]))
                for i in item[1]:
                    f.write(f';{i}')
                f.write('\n')
            
    
    return total_cycles

In [3]:

# Define the custom colormap's endpoints and colors
#lower_color = 'white'
#upper_color = 'white'

cmap = plt.cm.viridis
#cmap.set_under = [1,0,0]
#cmap.set_over = [0,1,0]
#cmap.set_under(lower_color)
#cmap.set_over(upper_color)
#print(cmap.colors)

path = 'rearrang_ts_2/'
name = 'test'

cycle_values = get_out_file_information(path, position_of_sulfur_in_xyz_file=20, max_opt_cycle=1000, save=name)
max_opt_cycle = 1000

color_1=np.array([coord[1] for coord in cycle_values.values()], dtype=float)
#print(cmap.colors)

cmap_1 = pmv.ColorMap([(0, np.array([1,0,0], dtype= float)), (1, cmap.colors[1]), ((max_opt_cycle-1), cmap.colors[-2]), (max_opt_cycle, np.array([0,1,0], dtype = float))])


label = pmv.Labels(np.array([coord[0] for coord in cycle_values.values()]), color_1, name=f'labels_{name}')


p = pmv.Points(np.array([coord[0] for coord in cycle_values.values()]), name=f'opt_cycle_{name}', color=color_1, 
                values_are_single_color=False, colormap=cmap_1)
print(p.color)
print(p.colormap._color_type)
p.colormap.name = f'opt_cycle_axis_{name}'
pmv.Group([p, p.colormap, label], f'{name}_opt_cycles_points').write(f'{eval_path}{name}_by_opt_cycle.py')

conf_69 has no opt cycles -> value set to 0
conf_68 has no opt cycles -> value set to 0
conf_4 has no opt cycles -> value set to 0
conf_80 has no opt cycles -> value set to 0
conf_101 has 1000 opt cycles
conf_38 has 1000 opt cycles
conf_39 has 1000 opt cycles
conf_49 has 1000 opt cycles
conf_12 has 1000 opt cycles
conf_70 has no opt cycles -> value set to 0
[ 350.    0.  273.  362.    0.   27.  220.  156.    0.    0. 1000. 1000.
 1000. 1000.   18.  163.   24.   47.  300.   41.   22.   20.  490.  586.
  258.   35.   43.   55.   87.   21.  446. 1000.    0.  771.]
segmented_single
